In [ ]:
import sys
import numpy as np
import pandas as pd
import dask.dataframe as dd
import math
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import time
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.decomposition import PCA
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
import collections
import optuna
from optuna.samplers import TPESampler

import feather
import os
import gc
import glob
import time

import lightgbm as lgb
from sklearn.linear_model import LogisticRegression

from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score,precision_recall_curve
from sklearn.metrics import make_scorer,f1_score, recall_score, accuracy_score, average_precision_score, auc
from sklearn.metrics import f1_score, mean_squared_error, r2_score, classification_report, confusion_matrix, accuracy_score, precision_score

import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
import matplotlib.style as style
style.use('fivethirtyeight')
import seaborn as sns
import pickle

import datatable as dt
from datatable import (dt, f, g, by, ifelse, update, sort, join,
                      count, min, max, mean, sum, rowsum)
import copy

In [ ]:
N_ROWS = 99271101

def get_index_np():
    return np.arange(N_ROWS)

In [ ]:
def load_data_np(file_path, columns_selected):
    data = np.load(file_path , allow_pickle=True)
    df = pd.DataFrame(index=get_index_np())
    for col in tqdm(columns_selected):
        df[col] = data[col]
    return df

In [ ]:
def load_data_np_multifiles(file_path, file_path2, file_path3, columns_selected, columns_selected2, columns_selected3):
    data = np.load(file_path , allow_pickle=True)
    data2 = np.load(file_path2 , allow_pickle=True)
    data3 = np.load(file_path3 , allow_pickle=True)
    df = pd.DataFrame(index=get_index_np())
    for col in tqdm(columns_selected):
        df[col] = data[col]
    for col in tqdm(columns_selected2):
        df[col] = data2[col]
    for col in tqdm(columns_selected3):
        df[col] = data3[col]
    return df

## Loading data

In [ ]:
%%time
#Check features data
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_features_final_new.npz'
index= ['user_id']

index_data = load_data_np(file_path, index )

In [ ]:
%%time
def get_train_val_idxs(TRAIN_SIZE, VAL_SIZE):
    train_idxs = []
    val_idxs = []
    NEW_USER_FRAC = 1/4 # fraction of new users in 
    np.random.seed(42)
    
    # create df with user_ids and indices
    df = pd.DataFrame(index=get_index_np())
    for col in ['user_id']:
        df[col] = index_data[col]

    df['index'] = df.index.values.astype(np.uint32)
    user_id_index = df.groupby('user_id')['index'].apply(np.array)
    
    # iterate over users in random order
    for indices in user_id_index.sample(user_id_index.size, random_state=42):
        if len(train_idxs) > TRAIN_SIZE:
            break

        # fill validation data
        if len(val_idxs) < VAL_SIZE:
            # add new user
            if np.random.rand() < NEW_USER_FRAC:
                val_idxs += list(indices)
            # randomly split user between train and val otherwise
            else:
                offset = np.random.randint(0, indices.size)
                train_idxs += list(indices[:offset])
                val_idxs += list(indices[offset:])
        else:
            train_idxs += list(indices)
        
    return train_idxs, val_idxs
train_idxs, val_idxs = get_train_val_idxs(int(100e6), 5e6)
#train_idxs, val_idxs = get_train_val_idxs(int(50e6), 2.5e6)
#train_idxs, val_idxs = get_train_val_idxs(int(10e6), 0.5e6)
print(f'len train_idxs: {len(train_idxs)}, len validation_idxs: {len(val_idxs)}')

In [ ]:
def load_data_np_multifiles(train_idxs,
                            val_idxs,
                            file_path,
                            file_path2,
                            file_path3,
                            file_path4,
                            file_path5,
                            file_path6,
                            file_path7,
                            file_path8,
                            file_path9,
                            file_path10,
                            columns_selected,
                            columns_selected2,
                            columns_selected3,
                            columns_selected4,
                            columns_selected5,
                            columns_selected6,
                            columns_selected7,
                            columns_selected8,
                            columns_selected9,
                            columns_selected10):
    
    data = np.load(file_path , allow_pickle=True)
    data2 = np.load(file_path2 , allow_pickle=True)
    data3 = np.load(file_path3 , allow_pickle=True)
    data4 = np.load(file_path4 , allow_pickle=True)
    data5 = np.load(file_path5 , allow_pickle=True)
    data6 = np.load(file_path6 , allow_pickle=True)
    data7 = np.load(file_path7 , allow_pickle=True)
    data8 = np.load(file_path8 , allow_pickle=True)
    data9 = np.load(file_path9 , allow_pickle=True)
    data10 = np.load(file_path10 , allow_pickle=True)
    #data11 = np.load(file_path11 , allow_pickle=True)
    
    X_train = np.ndarray(shape=(len(train_idxs), len(global_features)), dtype=np.float32)
    X_val = np.ndarray(shape=(len(val_idxs), len(global_features)), dtype=np.float32)
    for col in tqdm(columns_selected):
        X_train[:,global_features_dict.get(col)] = data[col][train_idxs].astype(np.float32)
        X_val[:,global_features_dict.get(col)] = data[col][val_idxs].astype(np.float32)
        
    for col in tqdm(columns_selected2):
        #print(col)
        #print(global_features_dict.get(col))
        X_train[:,global_features_dict.get(col)] = data2[col][train_idxs].astype(np.int8)
        X_val[:,global_features_dict.get(col)] = data2[col][val_idxs].astype(np.int8)
        
    for col in tqdm(columns_selected3):
        X_train[:,global_features_dict.get(col)] = data3[col][train_idxs].astype(np.int8)
        X_val[:,global_features_dict.get(col)] = data3[col][val_idxs].astype(np.int8)
        
    for col in tqdm(columns_selected4):
        X_train[:,global_features_dict.get(col)] = data4[col][train_idxs].astype(np.int8)
        X_val[:,global_features_dict.get(col)] = data4[col][val_idxs].astype(np.int8)
        
    for col in tqdm(columns_selected5):
        X_train[:,global_features_dict.get(col)] = data5[col][train_idxs].astype(np.int8)
        X_val[:,global_features_dict.get(col)] = data5[col][val_idxs].astype(np.int8)
        
    for col in tqdm(columns_selected6):
        X_train[:,global_features_dict.get(col)] = data6[col][train_idxs].astype(np.int8)
        X_val[:,global_features_dict.get(col)] = data6[col][val_idxs].astype(np.int8)
        
    for col in tqdm(columns_selected7):
        X_train[:,global_features_dict.get(col)] = data7[col][train_idxs].astype(np.int8)
        X_val[:,global_features_dict.get(col)] = data7[col][val_idxs].astype(np.int8)
        
    for col in tqdm(columns_selected8):
        X_train[:,global_features_dict.get(col)] = data8[col][train_idxs].astype(np.int8)
        X_val[:,global_features_dict.get(col)] = data8[col][val_idxs].astype(np.int8)
        
    for col in tqdm(columns_selected9):
        X_train[:,global_features_dict.get(col)] = data9[col][train_idxs].astype(np.int8)
        X_val[:,global_features_dict.get(col)] = data9[col][val_idxs].astype(np.int8)
        
    for col in tqdm(columns_selected10):
        X_train[:,global_features_dict.get(col)] = data10[col][train_idxs].astype(np.int8)
        X_val[:,global_features_dict.get(col)] = data10[col][val_idxs].astype(np.int8)
    
        # add the target
    y_train = data[target][train_idxs].astype(np.int8)
    y_val = data[target][val_idxs].astype(np.int8)
        
    print(f'X_train.shape: {X_train.shape}\t y_train.shape: {y_train.shape}')
    print(f'X_val.shape: {X_val.shape}\t y_val.shape: {y_val.shape}')
    return X_train, X_val, y_train, y_val

In [ ]:
%%time
#Check features data
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/data_features_final_new.npz'
file_path2 = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/questions_columns/question_historic_perf_attempt_full_1.npz'
file_path3 = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/questions_columns/question_historic_perf_attempt_full_2.npz'
file_path4 = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/questions_columns/question_historic_perf_attempt_full_3.npz'
file_path5 = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/questions_columns/question_historic_perf_attempt_full_4.npz'
file_path6 = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/questions_columns/question_historic_perf_attempt_full_5.npz'
file_path7 = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/questions_columns/question_historic_perf_attempt_full_6.npz'
file_path8 = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/questions_columns/question_historic_perf_attempt_full_7.npz'
file_path9 = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/questions_columns/question_historic_perf_attempt_full_8.npz'
file_path10 = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/questions_columns/question_historic_perf_attempt_full_9.npz'
#file_path11 = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/questions_columns/question_historic_perf_attempt_full_10.npz'


features = [
#'timestamp',
#'user_id',
'group',
'group2',
'content_id',
##'task_container_id',
'prior_question_elapsed_time',
##'bundle_id',
'part',
'previous_false_answer',
'content_answered_correctly_mean',
#'bundle_answered_correctly_mean',
'user_answered_correctly_mean',
'user_answered_correctly_count',
'hmean_user_content_accuracy',
#'content_answered_correctly_count',
#'answer_answered_correctly_mean',
##'question_answered_progress',
#'question_answered_progress_mean',
'answered_correctly_user',
#'lagtime_stats'
           ]


features2 = [

]

features3 = [

]

features4 = [

]

features5 = [

]

features6 = [

]

features7 = [

]

features8 = [

]

features9 = [

]

features10 = [

]

features11 = [

]

target= 'answered_correctly'

index_data= ['user_id']

columns = copy.deepcopy(features)
columns.append(target)


#Build features list
global_features = features + features2 + features3 + features4 + features5 + features6 + features7 + features8 + features9 + features10 
print(len(global_features))

global_features_dict = {k: v for v, k in enumerate(global_features)}

X_train, X_val, y_train, y_val = load_data_np_multifiles(train_idxs,
                    val_idxs,
                    file_path,
                    file_path2,
                    file_path3,
                    file_path4,
                    file_path5,
                    file_path6,
                    file_path7,
                    file_path8,
                    file_path9,
                    file_path10,              
                    features,
                    features2, 
                    features3,
                    features4, 
                    features5,
                    features6, 
                    features7,
                    features8,
                    features9,
                    features10
                              )


In [ ]:
%%time
import joblib
# save model
joblib.dump(X_train, 'X_train.pkl')
joblib.dump(X_val, 'X_val.pkl')
joblib.dump(y_train, 'y_train.pkl')
joblib.dump(y_val, 'y_val.pkl')

In [ ]:
categorical_feature = ['content_id', 'group', 'group2', 'bundle_id', 'part', 'tags', 'tags_label', 'prior_question_had_explanation', 'task_container_id']
categorical_feature_idxs = []
for v in categorical_feature:
    try:
        categorical_feature_idxs.append(features.index(v))
    except:
        pass

In [ ]:
categorical_feature_idxs

In [ ]:
# make train and validation dataset
train_data = lgb.Dataset(
    data = X_train,
    label = y_train,
    categorical_feature = None,
)

val_data = lgb.Dataset(
    data = X_val,
    label = y_val,
    categorical_feature = None,
)

In [ ]:
# NEW from: 
lgbm_params = {
    'objective': 'binary',
    'metric': ['auc'],
}

In [ ]:
features

In [ ]:
%%time
#13min
def train():
    evals_result = {}
    model = lgb.train(
        params = lgbm_params,
        train_set = train_data,
        valid_sets = [train_data, val_data],
        num_boost_round = 100,
        verbose_eval = 10,
        evals_result = evals_result,
        early_stopping_rounds = 10,
        categorical_feature = categorical_feature_idxs,
        feature_name = global_features
    )

    # save model
    model.save_model(f'/Users/nlisch/downloads/Kaggle_riid/process/prediction/model_final.lgb')
    import joblib
    # save model
    joblib.dump(model, 'lgb_model_final.pkl')
    return model, evals_result
    
model, evals_result = train()

In [ ]:
def plot_history(evals_result):
    for metric in ['auc']:
        plt.figure(figsize=(20,8))
        
        for key in evals_result.keys():
            history_len = len(evals_result.get(key)[metric])
            history = evals_result.get(key)[metric]
            x_axis = np.arange(1, history_len + 1)
            plt.plot(x_axis, history, label=key)
        
        x_ticks = list(filter(lambda e: (e % (history_len // 100 * 10) == 0) or e == 1, x_axis))
        plt.xticks(x_ticks, fontsize=12)
        plt.yticks(fontsize=12)

        plt.title(f'{metric.upper()} History of training', fontsize=18);
        plt.xlabel('EPOCH', fontsize=16)
        plt.ylabel(metric.upper(), fontsize=16)
        
        if metric in ['auc']:
            plt.legend(loc='upper left', fontsize=14)
        else:
            plt.legend(loc='upper right', fontsize=14)
        plt.grid()
        plt.show()

plot_history(evals_result)

In [ ]:
# plot the feature importance in terms of gain and split
def show_feature_importances(model, importance_type, max_num_features=10**10):
    feature_importances = pd.DataFrame()
    feature_importances['feature'] = global_features
    feature_importances['value'] = pd.DataFrame(model.feature_importance(importance_type))
    feature_importances = feature_importances.sort_values(by='value', ascending=False) # sort feature importance
    feature_importances.to_csv(f'feature_importances_{importance_type}.csv') # write feature importance to csv
    feature_importances = feature_importances[:50] # only show max_num_features
    
    plt.figure(figsize=(20, 8))
    plt.xlim([0, feature_importances.value.max()*1.1])
    plt.title(f'Feature {importance_type}', fontsize=18);
    sns.barplot(data=feature_importances, x='value', y='feature', palette='rocket');
    for idx, v in enumerate(feature_importances.value):
        plt.text(v, idx, "  {:.2e}".format(v))

show_feature_importances(model, 'gain')
show_feature_importances(model, 'split')

In [ ]:
# Check AUC
test = pd.DataFrame(X_val, columns=global_features)
y_real = y_val
y_proba = model.predict(test)
test['y_proba'] = y_proba
test['y_real'] = y_real
print('LGB score: ', roc_auc_score(test['y_real'], test['y_proba']))

In [ ]:
test.columns

In [ ]:
test[(test['timestamp'] == 0) & (test['content_id'] == 7900)][['timestamp', 'content_id',
       'prior_question_elapsed_time', 'part', 'previous_false_answer',
       'content_answered_correctly_mean', 'user_answered_correctly_mean', 'y_proba', 'y_real']]

In [ ]:
zoom = test.groupby(['part']).agg({'y_real': 'sum', 'y_proba':'sum' }).reset_index()
zoom

In [ ]:
zoom['auc'] = roc_auc_score(test['y_real'], test['y_proba'])
zoom

In [ ]:
import joblib
# save model
joblib.dump(model, 'lgb_model.pkl')

# SAVE TEST AND TRAIN

In [ ]:
train = pd.DataFrame(X_train, columns=global_features)
train[target] = y_train
test = pd.DataFrame(X_val, columns=global_features)
test[target] = y_val

In [ ]:
train.shape

In [ ]:
train.head(10)

In [ ]:
test.head(10)

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
%%time
#SAVE DATA
#Convert pandas columns to array
user_id = np.array(train['user_id'].tolist())
content_id = np.array(train['content_id'].tolist())
#task_container_id = np.array(train['task_container_id'].tolist())
answered_correctly = np.array(train['answered_correctly'].tolist())
prior_question_elapsed_time = np.array(train['prior_question_elapsed_time'].tolist())
#bundle_id = np.array(train['bundle_id'].tolist())
part = np.array(train['part'].tolist())
previous_false_answer = np.array(train['previous_false_answer'].tolist())
previous_correct_answer = np.array(train['previous_correct_answer'].tolist())
#content_answered_correctly_skew = np.array(train['content_answered_correctly_skew'].tolist())
content_answered_correctly_mean = np.array(train['content_answered_correctly_mean'].tolist())
#content_answered_correctly_std = np.array(train['content_answered_correctly_std'].tolist())
#user_answered_correctly_skew = np.array(train['user_answered_correctly_skew'].tolist())
user_answered_correctly_mean = np.array(train['user_answered_correctly_mean'].tolist())
#user_answered_correctly_std = np.array(train['user_answered_correctly_std'].tolist())
hmean_user_content_accuracy = np.array(train['hmean_user_content_accuracy'].tolist())

#hmean_answer_content_accuracy = np.array(train['hmean_answer_content_accuracy'].tolist())
#answer_answered_correctly_mean = np.array(train['answer_answered_correctly_mean'].tolist())
#content_answered_correctly_count = np.array(train['content_answered_correctly_count'].tolist())
user_answered_correctly_count = np.array(train['user_answered_correctly_count'].tolist())

answered_correctly_user = np.array(train['answered_correctly_user'].tolist())

question_6878 = np.array(train['question_6878'].tolist())
question_3731 = np.array(train['question_3731'].tolist())
question_4995 = np.array(train['question_4995'].tolist())
question_4414 = np.array(train['question_4414'].tolist())
question_1315 = np.array(train['question_1315'].tolist())
#question_6908 = np.array(train['question_6908'].tolist())
#question_6909 = np.array(train['question_6909'].tolist())
#question_6911 = np.array(train['question_6911'].tolist())
#question_6910 = np.array(train['question_6910'].tolist())
#question_877 = np.array(train['question_877'].tolist())
#question_3717 = np.array(train['question_3717'].tolist())
#question_4476 = np.array(train['question_4476'].tolist())
question_853 = np.array(train['question_853'].tolist())
#question_4108 = np.array(train['question_4108'].tolist())
question_951 = np.array(train['question_951'].tolist())
question_5177 = np.array(train['question_5177'].tolist())
#question_6318 = np.array(train['question_6318'].tolist())
#question_199 = np.array(train['question_199'].tolist())
#question_7898 = np.array(train['question_7898'].tolist())
#question_738 = np.array(train['question_738'].tolist())
#question_2594 = np.array(train['question_2594'].tolist())
#question_2595 = np.array(train['question_2595'].tolist())
question_2593 = np.array(train['question_2593'].tolist())
question_7218 = np.array(train['question_7218'].tolist())
question_7217 = np.array(train['question_7217'].tolist())
question_7216 = np.array(train['question_7216'].tolist())
#question_7219 = np.array(train['question_7219'].tolist())
question_10686 = np.array(train['question_10686'].tolist())
question_294 = np.array(train['question_294'].tolist())
#question_10685 = np.array(train['question_10685'].tolist())
#question_10688 = np.array(train['question_10688'].tolist())
question_10684 = np.array(train['question_10684'].tolist())
#question_10687 = np.array(train['question_10687'].tolist())
#question_3878 = np.array(train['question_3878'].tolist())
#question_6879 = np.array(train['question_6879'].tolist())
question_6877 = np.array(train['question_6877'].tolist())
#question_6880 = np.array(train['question_6880'].tolist())
question_6116 = np.array(train['question_6116'].tolist())
question_6173 = np.array(train['question_6173'].tolist())
question_4120 = np.array(train['question_4120'].tolist())
#question_175 = np.array(train['question_175'].tolist())
question_7876 = np.array(train['question_7876'].tolist())
question_7900 = np.array(train['question_7900'].tolist())
question_2065 = np.array(train['question_2065'].tolist())
question_2064 = np.array(train['question_2064'].tolist())
#question_2063 = np.array(train['question_2063'].tolist())
question_4492 = np.array(train['question_4492'].tolist())
#question_3365 = np.array(train['question_3365'].tolist())
question_3364 = np.array(train['question_3364'].tolist())
#question_3363 = np.array(train['question_3363'].tolist())
#question_4696 = np.array(train['question_4696'].tolist())
#question_1278 = np.array(train['question_1278'].tolist())
question_6370 = np.array(train['question_6370'].tolist())
#question_2946 = np.array(train['question_2946'].tolist())
#question_2947 = np.array(train['question_2947'].tolist())
question_answered_progress = np.array(train['question_answered_progress'].tolist()) 
question_answered_progress_mean = np.array(train['question_answered_progress_mean'].tolist())
#hmean_user_content_accuracy_progress_question = np.array(train['hmean_user_content_accuracy_progress_question'].tolist())

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/prediction/train_model_f2.npz',
                    #From data file
                    user_id = user_id,
                    content_id = content_id,
                    #task_container_id = task_container_id,
                    answered_correctly = answered_correctly,
                    prior_question_elapsed_time = prior_question_elapsed_time,
                    #bundle_id = bundle_id,
                    part = part,
                    previous_false_answer = previous_false_answer,
                    previous_correct_answer = previous_correct_answer,
                    #content_answered_correctly_skew = content_answered_correctly_skew,
                    content_answered_correctly_mean = content_answered_correctly_mean,
                    #content_answered_correctly_std = content_answered_correctly_std,
                    #user_answered_correctly_skew = user_answered_correctly_skew,
                    user_answered_correctly_mean = user_answered_correctly_mean,
                    #user_answered_correctly_std = user_answered_correctly_std,
                    hmean_user_content_accuracy = hmean_user_content_accuracy,
                    #hmean_answer_content_accuracy = hmean_answer_content_accuracy,
                    #answer_answered_correctly_mean = answer_answered_correctly_mean,
                    #content_answered_correctly_count = content_answered_correctly_count, 
                    user_answered_correctly_count = user_answered_correctly_count,
                    answered_correctly_user = answered_correctly_user,
                    
                    question_6878 = question_6878,
                    question_3731 = question_3731,
                    question_4995 = question_4995,
                    question_4414 = question_4414,
                    question_1315 = question_1315,
                    #question_6908 = question_6908,
                    #question_6909 = question_6909,
                    #question_6911 = question_6911,
                    #question_6910 = question_6910,
                    #question_877 = question_877,
                    #question_3717 = question_3717,
                    #question_4476 = question_4476,
                    question_853 = question_853,
                    #question_4108 = question_4108,
                    question_951 = question_951,
                    question_5177 = question_5177,
                    #question_6318 = question_6318,
                    #question_199 = question_199,
                    #question_7898 = question_7898,
                    #question_738 = question_738,
                    #question_2594 = question_2594,
                    #question_2595 = question_2595,
                    question_2593 = question_2593,
                    question_7218 = question_7218,
                    question_7217 = question_7217,
                    question_7216 = question_7216,
                    #question_7219 = question_7219,
                    question_10686 = question_10686,
                    question_294 = question_294,
                    #question_10685 = question_10685,
                    #question_10688 = question_10688,
                    question_10684 = question_10684,
                    #question_10687 = question_10687,
                    #question_3878 = question_3878,
                    #question_6879 = question_6879,
                    question_6877 = question_6877,
                    #question_6880 = question_6880,
                    question_6116 = question_6116,
                    question_6173 = question_6173,
                    question_4120 = question_4120,
                    #question_175 = question_175,
                    question_7876 = question_7876,
                    question_7900 = question_7900,
                    question_2065 = question_2065,
                    question_2064 = question_2064,
                    #question_2063 = question_2063,
                    question_4492 = question_4492,
                    #question_3365 = question_3365,
                    question_3364 = question_3364,
                    #question_3363 = question_3363,
                    #question_4696 = question_4696,
                    #question_1278 = question_1278,
                    question_6370 = question_6370,
                    #question_2946 = question_2946,
                    #question_2947 = question_2947,
                    question_answered_progress = question_answered_progress ,
                    question_answered_progress_mean = question_answered_progress_mean,
                    #hmean_user_content_accuracy_progress_question = hmean_user_content_accuracy_progress_question
                   )


In [ ]:
%%time
#SAVE DATA
#Convert pandas columns to array
user_id = np.array(test['user_id'].tolist())
content_id = np.array(test['content_id'].tolist())
#task_container_id = np.array(test['task_container_id'].tolist())
answered_correctly = np.array(test['answered_correctly'].tolist())
prior_question_elapsed_time = np.array(test['prior_question_elapsed_time'].tolist())
#bundle_id = np.array(test['bundle_id'].tolist())
part = np.array(test['part'].tolist())
previous_false_answer = np.array(test['previous_false_answer'].tolist())
previous_correct_answer = np.array(test['previous_correct_answer'].tolist())
#content_answered_correctly_skew = np.array(test['content_answered_correctly_skew'].tolist())
content_answered_correctly_mean = np.array(test['content_answered_correctly_mean'].tolist())
#content_answered_correctly_std = np.array(test['content_answered_correctly_std'].tolist())
#user_answered_correctly_skew = np.array(test['user_answered_correctly_skew'].tolist())
user_answered_correctly_mean = np.array(test['user_answered_correctly_mean'].tolist())
#user_answered_correctly_std = np.array(test['user_answered_correctly_std'].tolist())
hmean_user_content_accuracy = np.array(test['hmean_user_content_accuracy'].tolist())

#hmean_answer_content_accuracy = np.array(test['hmean_answer_content_accuracy'].tolist())
#answer_answered_correctly_mean = np.array(test['answer_answered_correctly_mean'].tolist())
#content_answered_correctly_count = np.array(test['content_answered_correctly_count'].tolist())
user_answered_correctly_count = np.array(test['user_answered_correctly_count'].tolist())

answered_correctly_user = np.array(test['answered_correctly_user'].tolist())

question_6878 = np.array(test['question_6878'].tolist())
question_3731 = np.array(test['question_3731'].tolist())
question_4995 = np.array(test['question_4995'].tolist())
question_4414 = np.array(test['question_4414'].tolist())
question_1315 = np.array(test['question_1315'].tolist())
#question_6908 = np.array(test['question_6908'].tolist())
#question_6909 = np.array(test['question_6909'].tolist())
#question_6911 = np.array(test['question_6911'].tolist())
#question_6910 = np.array(test['question_6910'].tolist())
#question_877 = np.array(test['question_877'].tolist())
#question_3717 = np.array(test['question_3717'].tolist())
#question_4476 = np.array(test['question_4476'].tolist())
question_853 = np.array(test['question_853'].tolist())
#question_4108 = np.array(test['question_4108'].tolist())
question_951 = np.array(test['question_951'].tolist())
question_5177 = np.array(test['question_5177'].tolist())
#question_6318 = np.array(test['question_6318'].tolist())
#question_199 = np.array(test['question_199'].tolist())
#question_7898 = np.array(test['question_7898'].tolist())
#question_738 = np.array(test['question_738'].tolist())
#question_2594 = np.array(test['question_2594'].tolist())
#question_2595 = np.array(test['question_2595'].tolist())
question_2593 = np.array(test['question_2593'].tolist())
question_7218 = np.array(test['question_7218'].tolist())
question_7217 = np.array(test['question_7217'].tolist())
question_7216 = np.array(test['question_7216'].tolist())
#question_7219 = np.array(test['question_7219'].tolist())
question_10686 = np.array(test['question_10686'].tolist())
question_294 = np.array(test['question_294'].tolist())
#question_10685 = np.array(test['question_10685'].tolist())
#question_10688 = np.array(test['question_10688'].tolist())
question_10684 = np.array(test['question_10684'].tolist())
#question_10687 = np.array(test['question_10687'].tolist())
#question_3878 = np.array(test['question_3878'].tolist())
#question_6879 = np.array(test['question_6879'].tolist())
question_6877 = np.array(test['question_6877'].tolist())
#question_6880 = np.array(test['question_6880'].tolist())
question_6116 = np.array(test['question_6116'].tolist())
question_6173 = np.array(test['question_6173'].tolist())
question_4120 = np.array(test['question_4120'].tolist())
#question_175 = np.array(test['question_175'].tolist())
question_7876 = np.array(test['question_7876'].tolist())
question_7900 = np.array(test['question_7900'].tolist())
question_2065 = np.array(test['question_2065'].tolist())
question_2064 = np.array(test['question_2064'].tolist())
#question_2063 = np.array(test['question_2063'].tolist())
question_4492 = np.array(test['question_4492'].tolist())
#question_3365 = np.array(test['question_3365'].tolist())
question_3364 = np.array(test['question_3364'].tolist())
#question_3363 = np.array(test['question_3363'].tolist())
#question_4696 = np.array(test['question_4696'].tolist())
#question_1278 = np.array(test['question_1278'].tolist())
question_6370 = np.array(test['question_6370'].tolist())
#question_2946 = np.array(test['question_2946'].tolist())
#question_2947 = np.array(test['question_2947'].tolist())
question_answered_progress = np.array(test['question_answered_progress'].tolist()) 
question_answered_progress_mean = np.array(test['question_answered_progress_mean'].tolist())
#hmean_user_content_accuracy_progress_question = np.array(test['hmean_user_content_accuracy_progress_question'].tolist())

In [ ]:
%%time
np.savez_compressed('/Users/nlisch/downloads/Kaggle_riid/process/prediction/test_model_f2.npz',
                    #From data file
                    user_id = user_id,
                    content_id = content_id,
                    #task_container_id = task_container_id,
                    answered_correctly = answered_correctly,
                    prior_question_elapsed_time = prior_question_elapsed_time,
                    #bundle_id = bundle_id,
                    part = part,
                    previous_false_answer = previous_false_answer,
                    previous_correct_answer = previous_correct_answer,
                    #content_answered_correctly_skew = content_answered_correctly_skew,
                    content_answered_correctly_mean = content_answered_correctly_mean,
                    #content_answered_correctly_std = content_answered_correctly_std,
                    #user_answered_correctly_skew = user_answered_correctly_skew,
                    user_answered_correctly_mean = user_answered_correctly_mean,
                    #user_answered_correctly_std = user_answered_correctly_std,
                    hmean_user_content_accuracy = hmean_user_content_accuracy,
                    #hmean_answer_content_accuracy = hmean_answer_content_accuracy,
                    #answer_answered_correctly_mean = answer_answered_correctly_mean,
                    #content_answered_correctly_count = content_answered_correctly_count, 
                    user_answered_correctly_count = user_answered_correctly_count,
                    answered_correctly_user = answered_correctly_user,
                    
                    question_6878 = question_6878,
                    question_3731 = question_3731,
                    question_4995 = question_4995,
                    question_4414 = question_4414,
                    question_1315 = question_1315,
                    #question_6908 = question_6908,
                    #question_6909 = question_6909,
                    #question_6911 = question_6911,
                    #question_6910 = question_6910,
                    #question_877 = question_877,
                    #question_3717 = question_3717,
                    #question_4476 = question_4476,
                    question_853 = question_853,
                    #question_4108 = question_4108,
                    question_951 = question_951,
                    question_5177 = question_5177,
                    #question_6318 = question_6318,
                    #question_199 = question_199,
                    #question_7898 = question_7898,
                    #question_738 = question_738,
                    #question_2594 = question_2594,
                    #question_2595 = question_2595,
                    question_2593 = question_2593,
                    question_7218 = question_7218,
                    question_7217 = question_7217,
                    question_7216 = question_7216,
                    #question_7219 = question_7219,
                    question_10686 = question_10686,
                    question_294 = question_294,
                    #question_10685 = question_10685,
                    #question_10688 = question_10688,
                    question_10684 = question_10684,
                    #question_10687 = question_10687,
                    #question_3878 = question_3878,
                    #question_6879 = question_6879,
                    question_6877 = question_6877,
                    #question_6880 = question_6880,
                    question_6116 = question_6116,
                    question_6173 = question_6173,
                    question_4120 = question_4120,
                    #question_175 = question_175,
                    question_7876 = question_7876,
                    question_7900 = question_7900,
                    question_2065 = question_2065,
                    question_2064 = question_2064,
                    #question_2063 = question_2063,
                    question_4492 = question_4492,
                    #question_3365 = question_3365,
                    question_3364 = question_3364,
                    #question_3363 = question_3363,
                    #question_4696 = question_4696,
                    #question_1278 = question_1278,
                    question_6370 = question_6370,
                    #question_2946 = question_2946,
                    #question_2947 = question_2947,
                    question_answered_progress = question_answered_progress ,
                    question_answered_progress_mean = question_answered_progress_mean,
                    #hmean_user_content_accuracy_progress_question = hmean_user_content_accuracy_progress_question
                   )


In [ ]:
N_ROWS = 99271101
N_ROWS_train = 50004223
N_ROWS_test = 2500267
N_ROWS_user_stats = 393656
N_ROWS_content_stats = 13523
N_ROWS_previous_stats_questions = 86867031
def get_index_np(type_data):
    if type_data == 'train':
        final = np.arange(N_ROWS_train)
    if type_data == 'test':
        final = np.arange(N_ROWS_test)
    if type_data == 'user_stats':
        final = np.arange(N_ROWS_user_stats)
    if type_data == 'content_stats':
        final = np.arange(N_ROWS_content_stats)
    if type_data == 'previous_stats_questions':
        final = np.arange(N_ROWS_previous_stats_questions)
    if type_data == 'full_dataset':
        final = np.arange(N_ROWS)
    return final

In [ ]:
def load_data_np(file_path, columns_selected, type_data):
    data = np.load(file_path , allow_pickle=True)
    df = pd.DataFrame(index=get_index_np(type_data))
    for col in tqdm(columns_selected):
        df[col] = data[col]
    return df

In [ ]:
%%time
#Check features data
file_path = '/Users/nlisch/downloads/Kaggle_riid/process/prediction/train_model_f.npz'
columns = ['answered_correctly_user']

data = load_data_np(file_path, columns, 'train' )

In [ ]:
train.shape

## Shappley value

In [ ]:
import shap
shap.initjs()

In [ ]:
test = pd.DataFrame(X_val, columns=global_features)

In [ ]:
%%time
# Create object that can calculate shap values
explainer = shap.TreeExplainer(model)

# Calculate Shap values
shap_values = explainer.shap_values(test[global_features])

#print('Shap Values shape : {}'.format(shap_values.shape))

In [ ]:
%%time
#Calculate dataframe of features importance following SHAP Values
shap_sum = np.abs(shap_values).mean(axis=0)

In [ ]:
%%time
importance_shap_values = pd.DataFrame([test[global_features].columns.tolist(), shap_sum.tolist()]).T

In [ ]:
%%time
importance_shap_values.columns = ['column_name', 'shap_importance']
importance_shap_values = importance_shap_values.sort_values('shap_importance', ascending=False)

In [ ]:
%%time
#Plot Features importance following SHAP Values
shap.summary_plot(shap_values, test[global_features], plot_type="bar", max_display=100)

In [ ]:
importance_shap_values.head(10)